# ExifTool
A Command Line tool to read and write file metadata.

Read [the documentation](https://exiftool.org/).

Although there's many online tools that allow you to read a file's metadata:
- https://exif.tools/
- https://exifmeta.com/
- https://www.metadata2go.com/
- https://onlineexifviewer.com/
- ... many more

these usually have one or more of the following limitations:
- works only for small files
- works only for image files
- works only on a single file
- does not allow writing metadata
- raises security concerns, especially if you want to extract the metadata for sensitive files

If none of the above is a limitation to your work, and many times it won't be, then you should use such online metadata extraction tool. If you do hit one or more of the limitations then the best thing to do is install and run ExifTool direclty.

If you use this notebook, none of the above limitations will be a problem (only the image size depending on where you run the notebook). The instructions here should also work on your local computer but ExifTool needs to be installed differently on different operating systems, see [installing](https://exiftool.org/install.html).

### Step 1 - install the tool on your system
These commands are meant for a Mac/Unix system and will work on Google Colab and Binder.

In [ ]:
!apt install libimage-exiftool-perl

If you get a `Permission denied` error that means you don't have installation permissions, if you do have those then you should be able to run the same command as a super user, and you'll need to enter your root credentials.

In [ ]:
!sudo apt install libimage-exiftool-perl

### Step 2 - get metadata for a file

By inspecting the metadata of a video on the ‘DNR’ People’s Militia telegram channel from February 18th 2022, Bellingcat [revealed](https://twitter.com/EliotHiggins/status/1495355366141534208) that the video was filmed days before the stated date (see article [here](https://www.bellingcat.com/news/2022/02/23/documenting-and-debunking-dubious-footage-from-ukraines-frontlines/)). 

In this example, we're downloading an archived version of the video and replicating that same analysis.

The post in question is still available at time of writing so you can also go and download it directly from Telegram at https://t.me/nm_dnr/6192 . Telegram is one of the few social media networks that does not erase metadata tags from uploaded media. Still, this approach can still be valid on websites, uploaded documents, and more. 

We use `wget` to download the file and name `dnr.mp4` (it's around 100MB).

In [ ]:
!wget "https://bellingcat-archive.nyc3.cdn.digitaloceanspaces.com/no-dups/0d32acf6bcc1577e31cc8784/909064ea499345b9b8b2fcbc.mp4" -O dnr.mp4 

We can now call `exiftool` for this file and see its metadata tags.
- left: the metadata tag name
- right: the corresponding tag value

  The first few entries such as `ExifTool Version Number` actually describe the environment and file location/name. 

You should read through the dozens of tags and analyse them, but do note:
- `File Modification/Access` date and times pertain to your copy of the file
- `Create Date`/`Media Create Date` are the original file's metadata and they point to `2022:02:08 10:29:30` that's 10 days before the claimed date

This file contains many metadata tags from the video editing process:
- `Pantry History Software Agent   : Adobe Premiere Pro CC 2018.1 (Windows)` describing software used
- `Windows Atom Unc Project Path   : \\?\M:\Проекты\2022\02 Февраль\2022-02-08 ДРГ++.prproj` describing the file path where the video project was saved
- `Ingredients File Path           : 2021-02-04 ВИДЕО-ЗАПИСЬ ДРГ(+).mp4, 2021-02-04 ВИДЕО-ЗАПИСЬ ДРГ(+).mp4` ingredients aka other videos that were used to make the final video, using these names you can actually find the source material taken from a YouTube video of Finnish military exercises.

In [ ]:
!exiftool dnr.mp4

Let's extract only metadata that has the word `date` in it. 

In [ ]:
!exiftool "-*date*" dnr.mp4

It is usually a good idea to ask ExifTool for duplicate metadata tags (`-a`) along undefined ones (`-u`) as these can contain additional information, let's do that for the above command.

You will notice many more entries, this is because only the latest non-duplicate metadata tags are showed by default. To analyse `dnr.mp4` you should call it with `-a` and `-u` and without any filter so you can see the chai of recorded video editing events. 

In [ ]:
!exiftool "-*date*" -a -u dnr.mp4

### Step 3 - extract and search metadata from many files
You can call ExifTool to extract the metadata tags of many files at the same time, ideally by having them all in the same folder or set of subfolders. You can additionally call it only for some file extensions or file name patterns.

As an example we're creating 10 empty image files in an `images` folder, you could of course upload a folder or other documents.

In [ ]:
# creates 10 images inside the images/ folder
!mkdir images; touch images/image{1..10}.jpg

Below, we're calling exiftool for all JPG files (`-ext jpg`) recursively inside the images folder (`-r ./images`). 

In [ ]:
!exiftool -ext jpg -r ./images

Now, we're doing the same filter but saving the results in CSV format and writing it to a `jpg_metadata.csv` file, you can then download and open it with either a text editor or a spreadsheets program for further analysis.

In [ ]:
!exiftool -csv -ext jpg -r ./images > jpg_metadata.csv

### Step 4 - erase all metadata
Sometimes it is useful to anonymise files by stripping off their metadata, ExifTool can do that for single files or even in batch.

Let's create a copy of the dnr.mp4 file and remove all its editable metadata (by setting all tags to empty with `-all=`).

Unless you use the `-overwrite_original` flag, ExifTool will move the original to a new filename ending in `_original`. If you do use the flag the metadata becomes irretrievable in your copy. 

In [ ]:
# copy to a new file
!cp dnr.mp4 dnr_copy.mp4

In [ ]:
# remove all metadata (ExifTool saves the original file to dnr_copy.mp4_original)
!exiftool -all= dnr_copy.mp4

In [ ]:
# this will remove all metadata and overwrite the dnr_copy.mp4 file
!exiftool -all= -overwrite_original dnr_copy.mp4

In [ ]:
# confirm that the relevant metadata has been removed
!exiftool dnr_copy.mp4

NB: Remember that metadata can be manually edited and its values should be cross-referenced with other open sources for increasing the certainty of a verification. 